### Задание на Семинар 5

**НЕ изменяйте и не удаляйте ячейки, написанные с помощью markdown, только ячейки с кодом**


In [ ]:
! pip install mysql-connector -q
! pip install pymysql -q

* Импортируйте библиотеки для работы с mysql
* Подключитесь к удаленному серверу используя для доступа учетные данные с [этой страницы](https://sss.krasilnikov.spb.ru/classsql). Объект для подключения поместите в переменную db_connection

(**1 балл**)

In [ ]:
import json
import mysql.connector as sql
import pandas as pd
import requests
import sqlalchemy
from termcolor import colored
from time import time

mysql_config = {
    'host': 'krasilnikov.spb.ru',
    'port': 3360,
    'user': '4049f890b5f59ee17a2554a1e271cc46',
    'password': '4049f890b5f59ee17a2554a1e271cc46',
    'database': '4049f890b5f59ee17a2554a1e271cc46_s1'
}

def timeit(func):
    def wrapper(*args, **kwargs):
        start_time = time()
        result = func(*args, **kwargs)
        print(colored('Execution time: %ds' % round(time() - start_time), 'green'))
        return result
    return wrapper

def with_connection(func):
    def wrapper(*args, **kwargs):
        connection = sql.connect(**mysql_config)
        try:
            result = func(connection, *args, **kwargs)
        except Exception as error:
            print(error)
        finally:
            connection.close()
        return result
    return wrapper

@timeit
@with_connection
def execute(connection, query):
    cursor = connection.cursor(buffered=True)
    results = None
    print(colored(query, 'blue'))
    try:
        cursor.execute(query)
        connection.commit()
        if cursor.rowcount:
            results = cursor.fetchall()
    except Exception as error:
            print(error)
    finally: 
        cursor.close()
    return results

execute('SELECT 1')

SELECT 1
Execution time: 2s


[(1,)]

* Выполните загрузку данных из [файла](https://sss.krasilnikov.spb.ru/images/files/class_uploader/4/s1.json) в объект json

(**1 балл**)

In [ ]:
url = 'https://sss.krasilnikov.spb.ru/images/files/class_uploader/4/s1.json'
json = json.loads(requests.get(url).text)
print(len(json['DoctorAUS']))

5191


* Создайте на SQL сервере таблицу DoctorAUS c полями, соответствующими данным в объекте. Выбор типа столбов осуществите самостоятельно основываясь на данных (используйте типы только из тех, что были рассмотрены на лекции). Название столбцов возьмите из json объекта.

(**3 балла**)

In [ ]:
def check_table_exists(table_name):
    tables_exist = execute('SHOW TABLES;')
    return tables_exist and (table_name, ) in tables_exist
        
def create_table(table_name, schema):
    def column_statement(column): 
        types = {int: 'INT', float: 'FLOAT', str: 'VARCHAR(255)'}
        return f'  {column} {types[schema[column]]}'
    
    if check_table_exists(table_name): 
        execute(f'DROP TABLE {table_name};')
    columns = ',\n'.join(column_statement(col) for col in schema.keys())
    create_statement = f'CREATE TABLE {table_name} (\n{columns}\n);'
    execute(create_statement)

doctor_aus = 'DoctorAUS'
schema = {
    'actdays': int,
    'age': float,
    'id': int,
    'illness': int,
    'insurance': str,
    'sex': int
}

create_table(doctor_aus, schema)

SHOW TABLES;
Execution time: 2s
DROP TABLE DoctorAUS;
Execution time: 2s
CREATE TABLE DoctorAUS (
  actdays INT,
  age FLOAT,
  id INT,
  illness INT,
  insurance VARCHAR(255),
  sex INT
);
Execution time: 2s


* Заполните данными таблицу DoctorAUS на SQL сервере 

(**2 балла**)

In [ ]:
@timeit
def to_sql(df, table_name, mysql_config):
    sqlalchemy_config = {**{'drivername': 'mysql+pymysql'}, **mysql_config}
    sqlalchemy_config['username'] = sqlalchemy_config.pop('user')
    try:
        engine = sqlalchemy.create_engine(sqlalchemy.engine.url.URL(**sqlalchemy_config))
        connection = engine.connect()
        if check_table_exists(table_name): 
            execute(f'TRUNCATE TABLE {table_name};')
        df.to_sql(table_name, connection, index=False, if_exists='append')
    except Exception as error:
        print(error)
    finally:
        connection.close()

df = pd.DataFrame.from_dict(json[doctor_aus]).dropna()
to_sql(df, doctor_aus, mysql_config)
execute(f'SELECT count(*) from {doctor_aus}')

SHOW TABLES;
Execution time: 2s
TRUNCATE TABLE DoctorAUS;
Execution time: 2s
Execution time: 10s
SELECT count(*) from DoctorAUS
Execution time: 2s


[(5190,)]

 * Создайте таблицу DoctorContacts аналогично предыдущим двум заданиям.
  * дополнительно самым первым создайте поле id c возрастающим ключом auto_increment;
  * пропустите колонку с пустым названием.
 * Заполните данными таблицу DoctorContacts.
 
(**2 балла**)

In [ ]:
doctor_contacts = 'DoctorContacts'
schema = {
    'mdu': int,
    'lc': int,
    'lpi': float,
    'fmde': int,
    'physlim': int,
}

create_table(doctor_contacts, schema)
print(len(json[doctor_contacts]))
df = pd.DataFrame.from_dict(json[doctor_contacts], orient='index').dropna().drop('', axis=1)
df['physlim'] = df['physlim'].astype(bool).astype(int)
to_sql(df, doctor_contacts, mysql_config)
execute(f'SELECT count(*) from {doctor_contacts}')

SHOW TABLES;
Execution time: 2s
DROP TABLE DoctorContacts;
Execution time: 2s
CREATE TABLE DoctorContacts (
  mdu INT,
  lc INT,
  lpi FLOAT,
  fmde INT,
  physlim INT
);
Execution time: 2s
20187
SHOW TABLES;
Execution time: 2s
TRUNCATE TABLE DoctorContacts;
Execution time: 2s
Execution time: 11s
SELECT count(*) from DoctorContacts
Execution time: 2s


[(20186,)]

* Выгрузите в датафрейм data данные 
 * по столбцам illness и actdays
 * из таблицы DoctorAUS
 * с переменной age от 0.19 до 0.28
 * с переменной insurance = levyplus

* По окончании загрузки отсортируйте датафрейм сначала по столбцу actdays, затем по illness, по убыванию. 

(**3 балла**)

In [ ]:
datas = execute("""
SELECT 
  actdays,
  illness
FROM DoctorAUS
WHERE 
  age BETWEEN 0.19 AND 0.28
  AND insurance = 'levyplus'
""")
columns = ['actdays', 'illness']
data = pd.DataFrame(datas, columns=columns).sort_values(columns, ascending=[True, False])
data.head()